In [2]:
import pandas as pd
import google.generativeai as genai
import time
import os
import json
from dotenv import load_dotenv

# ==========================================
# 1. CONFIGURAÇÃO (IGNORANDO WARNINGS)
# ==========================================
print("🚀 Configurando Motor de IA (Gemini 2.0)...")

# Carrega .env
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError("❌ ERRO: Chave GEMINI_API_KEY não encontrada no arquivo .env")

genai.configure(api_key=api_key)

# Modelo validado (Flash 2.0)
NOME_MODELO = 'gemini-2.0-flash'

try:
    model = genai.GenerativeModel(NOME_MODELO)
    print(f"✅ Modelo autenticado: {NOME_MODELO}")
except Exception as e:
    print(f"⚠️ Erro ao carregar {NOME_MODELO}, tentando fallback...")
    model = genai.GenerativeModel('gemini-pro')

# ==========================================
# 2. CARGA DOS DADOS
# ==========================================
DATA_PATH = '../data/processed/relatorio_final_gaps.xlsx'

try:
    df_gaps = pd.read_excel(DATA_PATH)
    df_processar = df_gaps.copy()
    print(f"📂 Carregados {len(df_processar)} itens para auditoria.")
except FileNotFoundError:
    raise FileNotFoundError("Arquivo não encontrado. Rode o notebook 3.0 novamente.")

# ==========================================
# 3. ENGENHARIA DE PROMPT
# ==========================================
def consultar_gemini(pergunta_tecnica, nota_perdida, cidade_referencia):
    
    prompt = f"""
    ATUE COMO: Auditor Sênior de Contas Públicas (TCE/RJ) e Especialista em LAI.
    
    O CASO:
    O município de Paracambi-RJ perdeu {nota_perdida} pontos no item: "{pergunta_tecnica}".
    COMPARATIVO: A cidade de {cidade_referencia} atende a este requisito.

    TAREFA:
    Gere um parecer técnico curto em JSON ÚNICO (não use listas).
    Seja rigoroso com o risco jurídico.

    OUTPUT ESPERADO (JSON):
    {{
        "gravidade": "Alta, Média ou Baixa",
        "risco_legal": "Explique a consequência (multa, rejeição de contas, improbidade)",
        "acao_corretiva": "Ação técnica para o site (ex: criar aba específica, publicar CSV)",
        "base_legal_citada": "Cite a Lei (LRF, LAI) e Artigo se possível"
    }}
    """
    
    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                response_mime_type="application/json"
            )
        )
        return response.text
    except Exception as e:
        # Se der erro na API, retorna um JSON de erro válido para não quebrar o loop
        return json.dumps({"risco_legal": f"Erro API: {str(e)}", "gravidade": "Erro"})

# ==========================================
# 4. PROCESSAMENTO BLINDADO
# ==========================================
print("🤖 Iniciando varredura jurídica...")

resultados_ia = []

for index, row in df_processar.iterrows():
    pergunta = row['Questão Chave']
    perda = row['Pontos Perdidos']
    ref = row.get('Referência (Benchmarking)', 'Não informado')
    
    print(f"   [{index+1}/{len(df_processar)}] Auditando: {pergunta[:40]}...")
    
    # Chamada IA
    json_response = consultar_gemini(pergunta, perda, ref)
    
    # Objeto padrão caso falhe tudo
    dados_processados = {
        'Gravidade': 'N/A',
        'Risco Legal': 'Falha na Análise',
        'Ação Recomendada': 'Verificar Manualmente',
        'Lei': ''
    }

    try:
        # Limpeza preventiva
        clean_response = json_response.replace('```json', '').replace('```', '').strip()
        dados = json.loads(clean_response)
        
        # --- AQUI ESTÁ A CORREÇÃO DO ERRO ---
        # Se o Gemini devolveu uma lista [{...}], pegamos o primeiro item [0]
        if isinstance(dados, list):
            if len(dados) > 0:
                dados = dados[0]
            else:
                dados = {} 

        # Atualiza o objeto com os dados seguros
        dados_processados.update({
            'Gravidade': dados.get('gravidade', 'N/A'),
            'Risco Legal': dados.get('risco_legal', 'N/A'),
            'Ação Recomendada': dados.get('acao_corretiva', 'N/A'),
            'Lei': dados.get('base_legal_citada', '')
        })

    except json.JSONDecodeError:
        print(f"      ⚠️ JSON inválido na linha {index}. Salvando resposta bruta.")
        dados_processados['Risco Legal'] = f"Erro Parse: {json_response[:50]}..."

    # Monta a linha final
    item_final = {
        'Questão Original': pergunta,
        'Pontos Perdidos': perda,
        'Referência': ref,
        **dados_processados # Desempacota o dicionário processado
    }
    
    resultados_ia.append(item_final)
    time.sleep(1) # Respeito ao Rate Limit

# ==========================================
# 5. EXPORTAÇÃO
# ==========================================
df_final = pd.DataFrame(resultados_ia)

CAMINHO_FINAL = '../data/processed/LAUDO_TECNICO_GEMINI_FINAL.xlsx'
df_final.to_excel(CAMINHO_FINAL, index=False)

print("\n✅ SUCESSO! O código corrigiu a lista automaticamente.")
print(f"📄 Relatório salvo em: {CAMINHO_FINAL}")

🚀 Configurando Motor de IA (Gemini 2.0)...
✅ Modelo autenticado: gemini-2.0-flash
📂 Carregados 9 itens para auditoria.
🤖 Iniciando varredura jurídica...
   [1/9] Auditando: 24 - PEDIDO 2: O pedido 2 foi enviado co...
   [2/9] Auditando: 25 - PEDIDO 3: O pedido 3 foi enviado co...
   [3/9] Auditando: 12 -  CONTRATOS: O ente federado disponi...
   [4/9] Auditando: 11 -  O ente federado disponibiliza cons...
   [5/9] Auditando: 14 - OBRAS PÚBLICAS: O ente federado dis...
   [6/9] Auditando: 13 - O ente federado permite gerar relat...
   [7/9] Auditando: 4 - UNIDADES ADMINISTRATIVAS: O ente fed...
   [8/9] Auditando: 9 - LICITAÇÕES: O ente federado disponib...
   [9/9] Auditando: 19 -  BASES DE DADOS ABERTOS: O ente fed...

✅ SUCESSO! O código corrigiu a lista automaticamente.
📄 Relatório salvo em: ../data/processed/LAUDO_TECNICO_GEMINI_FINAL.xlsx
